In [24]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])
import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
df = pd.read_csv('spacex_web_scraped.csv')
df['Launch site'].unique()
launch_sites = df['Launch site'].unique()
print(launch_sites)
#Task 1
site_coords = {
    'CCAFS SLC-40': [28.561857, -80.577366],     # Cape Canaveral
    'KSC LC-39A': [28.608058, -80.603956],       # Kennedy Space Center
    'VAFB SLC-4E': [34.632834, -120.610746],     # Vandenberg
    'Boca Chica': [25.997222, -97.156944]        # Boca Chica (Starbase)
}
# Create a map centered in the US
site_map = folium.Map(location=[28.5, -95], zoom_start=4)

# Add markers
for site, coord in site_coords.items():
    folium.Marker(
        location=coord,
        popup=site,
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

# Show the map
site_map

#Task 2
from js import fetch
import io
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

circle = folium.Circle(
        coordinate,
        radius=1000,
        color='#000000',  # black
        fill=True
    ).add_child(folium.Popup(site_name))  # popup shows launch site name
    
    # Use folium.Marker with DivIcon (your second example)
marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name
        )
    )
    
    # Add both to map
site_map.add_child(circle)
site_map.add_child(marker)

# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
    # Text label marker (optional)
marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    )
site_map.add_child(marker)
# Re-initialize map centered at NASA
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch
for i, row in spacex_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    site = row['Launch Site']
    outcome = row['class']  # 1 for success, 0 for failure
    
    # Set marker color
    marker_color = 'green' if outcome == 1 else 'red'
    outcome_label = 'Success' if outcome == 1 else 'Failure'
    
    # Create marker
    folium.Marker(
        location=[lat, lon],
        popup=f"{site} - {outcome_label}",
        icon=folium.Icon(color=marker_color, icon='rocket', prefix='fa')
    ).add_to(site_map)

# Show the map
site_map
spacex_df.tail(10)
marker_cluster = MarkerCluster()
# Create a new column 'marker_color' based on 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
# Initialize map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through and add markers
for i, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']} - {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'], icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Display the map
site_map


['CCAFS' 'VAFB' 'Cape Canaveral' 'KSC' 'CCSFS']


In [30]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Example coordinates (latitude, longitude)
launch_site = (28.561857, -80.577366)  # Launch site
railway = (28.5632, -80.5707)          # Nearby railway

distance = haversine_distance(*launch_site, *railway)
print(f"Distance: {distance:.2f} km")


Distance: 0.67 km


In [26]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [31]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [32]:
launch_site_lat = 28.561857
launch_site_lon = -80.577366

# Coordinates of closest coastline (example)
coastline_lat = 28.56367
coastline_lon = -80.57163

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"Distance to closest coastline: {distance_coastline:.2f} km")

Distance to closest coastline: 0.60 km


In [37]:
coastline_coord = (28.56367, -80.57163)
distance = 0.60 
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
    icon_size=(20,20), 
    icon_anchor=(0,0),
html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
)
)
distance_marker.add_to(site_map)

In [38]:
# Coordinates
launch_site_coord = (28.561857, -80.577366)
coastline_coord = (28.56367, -80.57163)

# Create a list of locations for the line
coordinates = [launch_site_coord, coastline_coord]

# Create the PolyLine object
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue', opacity=0.7)

# Add the PolyLine to the map
site_map.add_child(lines)


In [40]:
launch_site_coord = (28.561857, -80.577366)
closest_city_coord = (28.5721, -80.5851)  # example closest city coordinate
distance_to_city = 1.10  # distance in kilometers (example)
line = folium.PolyLine(locations=[launch_site_coord, closest_city_coord], weight=2, color='green', opacity=0.7)
site_map.add_child(line)